In [1]:
import pandas as pd
import glob
import numpy as np
import pathlib as pl

### 1) Combine all the pks datasets

In [2]:
PATH_TO_PROCESSED_DATA = pl.Path('../datasets/processed')

cases_basic_table_files = glob.glob('../datasets/raw/police_stats/pks20[0-9][0-9]CasesBasicTable_excel.xlsx')
cases_development_files = glob.glob('../datasets/raw/police_stats/pks20[0-9][0-9]CasesDevelopment_excel.xlsx')
suspects_files = glob.glob('../datasets/raw/police_stats/pks20[0-9][0-9]Suspects_excel.xlsx')
len(cases_basic_table_files), len(cases_development_files), len(suspects_files)

(10, 10, 10)

 - first combine all the disticnt tables from all the years to 3 tables throughout the years

#### 1.11 PKS basic table

In [3]:
df = pd.read_excel(cases_basic_table_files[0], skiprows=8, usecols='B:T', header=None,
              names=["offence or offence category", "recorded cases", "percentage of all cases", "attempts number", "attempts %", "< 20,000 inhabitants", "20,000 up to 100,000 inhabitants", "100,000 up to 500,000 inhabitants", "500,000 and more inhabitants",  "unknown inhabitants", "firearm threatened", "firearm shot", "clearance number", "clearance %", "suspects count", "male suspects", "female suspects", "non-German suspects number", "non-German suspects %"]) # for sure there is a way how to automate the extraction of the names from various rows, but this seems to be easier

df


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
def combine_pks_tables(fnames: list[str], names: list[str], skiprows: int, usecols: str) -> pd.DataFrame:
    table = pd.DataFrame()

    for i, fname in enumerate(fnames):
        print(fname)
        
        year = 2014 + i
        new_names = [f"{year} {n}" for n in names] 
        df_i = pd.read_excel(fname, skiprows=skiprows, usecols=usecols, header=None,
                  names=new_names)
        
        table = pd.concat([table, df_i], axis=1)
    
    return table



In [ ]:
basic_table = combine_pks_tables(cases_basic_table_files, ["offence or offence category", "recorded cases", "percentage of all cases", "attempts number", "attempts %", "< 20,000 inhabitants", "20,000 up to 100,000 inhabitants", "100,000 up to 500,000 inhabitants", "500,000 and more inhabitants",  "unknown inhabitants", "firearm threatened", "firearm shot", "clearance number", "clearance %", "suspects count", "male suspects", "female suspects", "non-German suspects number", "non-German suspects %"], 8, 'B:T')

basic_table.to_csv(PATH_TO_PROCESSED_DATA / 'pks_basic_table.csv')

basic_table.shape 

../datasets/raw/police_stats\pks2014CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2015CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2016CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2017CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2018CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2019CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2020CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2021CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2022CasesBasicTable_excel.xlsx
../datasets/raw/police_stats\pks2023CasesBasicTable_excel.xlsx


(1139, 190)

#### 1.2 PKS Cases Development

In [ ]:
df = pd.read_excel(cases_development_files[0], skiprows=16, usecols='C,D,H', header=None, names=["offence or offence category", "recorded cases", "clearance rate %"])
df

,offence or offence category,recorded cases,clearance rate %
0,total offences,6082064,54.9
1,offences against life,2962,93.2
2,murder (sect. 211 PC),664,95.3
3,other types of murder,602,95.5
4,robbery attended with murder,44,93.2
...,...,...,...
1024,environmental offences pursuant to supplementa...,14890,75.3
1025,street crime,1342905,16.5
1026,damage to property by graffiti in total,95160,18.8
1027,illegal entry/illegal stay under the Aliens Ac...,135743,99.8


In [4]:
# for sure there is a way how to automate the extraction of the names from various rows, but this seems to be easier
cases_development_table = combine_pks_tables(cases_basic_table_files, ["offence or offence category", "recorded cases", "clearance rate %"], 16, 'C,D,H')

cases_development_table.to_csv(PATH_TO_PROCESSED_DATA / 'pks_development_table.csv')

cases_development_table.shape

NameError: name 'combine_pks_tables' is not defined

#### 1.3 Suspects